### Aufgabe 1 - Daten laden

In [21]:
import urllib

def getTextLower(url):
    response = urllib.request.urlopen(url)
    raw = response.read().decode('utf-8-sig')
    raw_lowered = raw.lower().strip()
    
    return raw_lowered

In [22]:
reviews = getTextLower('https://raw.githubusercontent.com/noxthot/deep-learning-v2-pytorch/master/sentiment-analysis-network/reviews.txt')

In [23]:
labels = getTextLower('https://raw.githubusercontent.com/noxthot/deep-learning-v2-pytorch/master/sentiment-analysis-network/labels.txt')

In [24]:
reviews_list = reviews.split('\n') # Split nach jedem Zeilenumbruch
labels_list = labels.split('\n')

In [25]:
len(reviews_list)

25000

In [26]:
len(labels_list)

25000

### Aufgabe 2 - Datenvorverarbeitung

In [27]:
import collections
from nltk.stem import PorterStemmer

ps = PorterStemmer()

cnt_positives = collections.Counter()
cnt_total = collections.Counter()
cnt = 0 # Diese Variable wird lediglich für eine Fortschriftsanzeige während der (langen) Ausführung dieser Codezelle verwendet.

for review, label in zip(reviews_list, labels_list):
    cnt += 1
    
    if cnt % 500 == 0: # Gib jede 500. Iteration eine Meldung am Bildschirm aus
        print(f"{cnt}. iteration of {len(reviews_list)}")
    
    good_review = (label == 'positive')
    
    for word in review.split():
        word_stemmed = ps.stem(word)        
        cnt_total[word_stemmed] += 1
        
        if good_review:
            cnt_positives[word_stemmed] += 1

500. iteration of 25000
1000. iteration of 25000
1500. iteration of 25000
2000. iteration of 25000
2500. iteration of 25000
3000. iteration of 25000
3500. iteration of 25000
4000. iteration of 25000
4500. iteration of 25000
5000. iteration of 25000
5500. iteration of 25000
6000. iteration of 25000
6500. iteration of 25000
7000. iteration of 25000
7500. iteration of 25000
8000. iteration of 25000
8500. iteration of 25000
9000. iteration of 25000
9500. iteration of 25000
10000. iteration of 25000
10500. iteration of 25000
11000. iteration of 25000
11500. iteration of 25000
12000. iteration of 25000
12500. iteration of 25000
13000. iteration of 25000
13500. iteration of 25000
14000. iteration of 25000
14500. iteration of 25000
15000. iteration of 25000
15500. iteration of 25000
16000. iteration of 25000
16500. iteration of 25000
17000. iteration of 25000
17500. iteration of 25000
18000. iteration of 25000
18500. iteration of 25000
19000. iteration of 25000
19500. iteration of 25000
20000.

In [28]:
cnt_total.most_common(10)

[('the', 336713),
 ('.', 327192),
 ('and', 164115),
 ('a', 163010),
 ('of', 145884),
 ('to', 135740),
 ('is', 107328),
 ('it', 104629),
 ('br', 101872),
 ('in', 94016)]

In [29]:
cnt_positives.most_common(10)

[('the', 173324),
 ('.', 159654),
 ('and', 89723),
 ('a', 83688),
 ('of', 76862),
 ('to', 66762),
 ('is', 57245),
 ('it', 52647),
 ('in', 50232),
 ('br', 49235)]

### Aufgabe 3 - Training

In [41]:
MIN_OCCURENCES = 50  # Minimum Vorkommnisse eines Wortstammes um als Trendwort berücksichtig werden zu können. Schützt uns vor seltenen Wörtern / Ausreißern.
MIN_TRENDVAL = 0.3   # Der Absolutwert des Trendwerts muss diesen Wert überschreiten um berücksichtigt zu werden. Schützt uns bspw. vor Stopwords.

trendwords = collections.Counter()

for word_stemmed in cnt_total:
    trendval = cnt_positives[word_stemmed] / cnt_total[word_stemmed]
    trendval_trans = (trendval - 0.5) * 2
    
    if cnt_total[word_stemmed] >= MIN_OCCURENCES and abs(trendval_trans) > MIN_TRENDVAL:
        trendwords[word_stemmed] = trendval_trans

In [42]:
trendwords.most_common(20)

[('goldsworthi', 1.0),
 ('antwon', 1.0),
 ('gunga', 1.0),
 ('gypo', 1.0),
 ('flavia', 1.0),
 ('yokai', 1.0),
 ('din', 0.9761904761904763),
 ('edi', 0.9646017699115044),
 ('iturbi', 0.962962962962963),
 ('gundam', 0.9619047619047618),
 ('visconti', 0.9615384615384615),
 ('feinston', 0.96),
 ('capot', 0.9375),
 ('felix', 0.9338842975206612),
 ('deanna', 0.9310344827586208),
 ('hayworth', 0.9310344827586208),
 ('fido', 0.9285714285714286),
 ('vanc', 0.9259259259259258),
 ('sammo', 0.9259259259259258),
 ('adel', 0.9259259259259258)]

In [43]:
trendwords.most_common()[::-1][:20]

[('boll', -0.9862068965517241),
 ('uwe', -0.9803921568627451),
 ('thunderbird', -0.974025974025974),
 ('beowulf', -0.9672131147540983),
 ('wayan', -0.9649122807017544),
 ('dahmer', -0.9636363636363636),
 ('gram', -0.9622641509433962),
 ('seagal', -0.9393939393939394),
 ('gamera', -0.9298245614035088),
 ('dreck', -0.9259259259259259),
 ('mst', -0.9),
 ('stinker', -0.8974358974358975),
 ('flimsi', -0.896551724137931),
 ('dilling', -0.8867924528301887),
 ('unwatch', -0.8833333333333333),
 ('unfunni', -0.8731343283582089),
 ('ugh', -0.8620689655172413),
 ('blah', -0.8571428571428572),
 ('yawn', -0.851063829787234),
 ('turd', -0.8484848484848485)]

### Aufgabe 4 - Test

In [50]:
myReview = input("INSERT YOUR REVIEW HERE (ENGLISH): ")

trendvals = 0

for word in myReview.split():
    word_stemmed = word.lower()
    
    trendvals += trendwords[word_stemmed]
    
if trendvals > 0:
    print(f"Dein Review war POSITIV mit Trendwert {trendvals}.")
else:
    print(f"Dein Review war NEGATIV mit Trendwert {trendvals}.")

INSERT YOUR REVIEW HERE (ENGLISH):  Best movie I've ever seen, though actors could perform better


Dein Review war POSITIV mit Trendwert 0.6681545449074817.


In [51]:
# Alternative Implementierung mit mehr Aussagekraft:
myReview = input("INSERT YOUR REVIEW HERE (ENGLISH): ")

trendvals = 0
cnt_trendwords = 0

for word in myReview.split():
    word_stemmed = word.lower()
    
    if trendwords[word_stemmed] != 0:
        trendvals += trendwords[word_stemmed]
        cnt_trendwords += 1
    
trendvals /= cnt_trendwords # Dividiere durch die Anzahl der vorhandenen Trendwörter. Dadurch liegt der Wert wieder im Intervall [-1, 1].

CONFIDENCE_TRENDVAL = 0.3

if trendvals > CONFIDENCE_TRENDVAL:
    print(f"Dein Review war POSITIV mit Trendwert {trendvals} basierend auf {cnt_trendwords} Trendwörtern.")
elif trendvals < CONFIDENCE_TRENDVAL:
    print(f"Dein Review war NEGATIV mit Trendwert {trendvals} basierend auf {cnt_trendwords} Trendwörtern.")
else:
    print(f"Dein Review kann nicht eindeutig eingeschätzt werden (Trendwert {trendvals} basierend auf {cnt_trendwords} Trendwörtern).")

INSERT YOUR REVIEW HERE (ENGLISH):  Best movie I've ever seen, though actors could perform better


Dein Review war POSITIV mit Trendwert 0.33407727245374086 basierend auf 2 Trendwörtern.
